In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

对weight initialization的理解还是不够，可能得看了论文才能搞得比较明白；2019-0923先放过这一块；实际使用先按业界公认的bestpractise去做，至于为什么，等有空了再细细研究。目前先把第一刷刷完。

## Does nn.Conv2d init work well?

[Jump_to lesson 9 video](https://course.fast.ai/videos/?lesson=9&t=21)

In [2]:
#export
from exp.nb_02 import *

def get_data():
    path = datasets.download_data(MNIST_URL, ext='.gz')
    with gzip.open(path, 'rb') as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
    return map(tensor, (x_train,y_train,x_valid,y_valid))

def normalize(x, m, s): return (x-m)/s

In [3]:
torch.nn.modules.conv._ConvNd.reset_parameters??

In [4]:
x_train,y_train,x_valid,y_valid = get_data()
train_mean,train_std = x_train.mean(),x_train.std()
x_train = normalize(x_train, train_mean, train_std)
x_valid = normalize(x_valid, train_mean, train_std)

In [5]:
x_train.shape

torch.Size([50000, 784])

In [6]:
x_train = x_train.view(-1,1,28,28)
x_valid = x_valid.view(-1,1,28,28)
x_train.shape,x_valid.shape

(torch.Size([50000, 1, 28, 28]), torch.Size([10000, 1, 28, 28]))

In [7]:
n,*_ = x_train.shape
c = y_train.max()+1
nh = 32
n,c

(50000, tensor(10))

In [8]:
l1 = nn.Conv2d(1, nh, 5)

In [9]:
x = x_valid[:100]

In [10]:
x.shape

torch.Size([100, 1, 28, 28])

In [11]:
def stats(x): return x.mean(),x.std()

In [12]:
l1.weight.shape

torch.Size([32, 1, 5, 5])

In [13]:
stats(l1.weight),stats(l1.bias)

((tensor(-0.0069, grad_fn=<MeanBackward0>),
  tensor(0.1150, grad_fn=<StdBackward0>)),
 (tensor(0.0088, grad_fn=<MeanBackward0>),
  tensor(0.1111, grad_fn=<StdBackward0>)))

In [14]:
t = l1(x)

In [15]:
stats(t)

(tensor(-0.0084, grad_fn=<MeanBackward0>),
 tensor(0.6707, grad_fn=<StdBackward0>))

In [16]:
init.kaiming_normal_(l1.weight, a=1.)
stats(l1(x))

(tensor(0.0019, grad_fn=<MeanBackward0>),
 tensor(1.1113, grad_fn=<StdBackward0>))

In [17]:
import torch.nn.functional as F

In [18]:
def f1(x,a=0): return F.leaky_relu(l1(x),a)

In [19]:
stats(x)

(tensor(-0.0363), tensor(0.9602))

In [20]:
init.kaiming_normal_(l1.weight, a=0)
stats(f1(x))

(tensor(0.5132, grad_fn=<MeanBackward0>),
 tensor(0.8649, grad_fn=<StdBackward0>))

上面说明，如果con2d后面跟着的是参数为0的leaky_relu，那么con2d里面的weight就用参数为0的kaiming_normal_，就能保持住variance

In [21]:
l1 = nn.Conv2d(1, nh, 5)
stats(f1(x))

(tensor(0.2116, grad_fn=<MeanBackward0>),
 tensor(0.3672, grad_fn=<StdBackward0>))

In [22]:
l1.weight.shape

torch.Size([32, 1, 5, 5])

In [23]:
# receptive field size
rec_fs = l1.weight[0,0].numel()
rec_fs

25

In [24]:
nf,ni,*_ = l1.weight.shape
nf,ni

(32, 1)

In [25]:
fan_in  = ni*rec_fs
fan_out = nf*rec_fs
fan_in,fan_out

(25, 800)

In [26]:
def gain(a): return math.sqrt(2.0 / (1 + a**2))

这个gain应该就是给weightinitialization多出的一个乘子；比如a=1时，那么实际上就是relu不发生，这时weight不需要额外的乘子，实际上gain(1)=1就是不给任何乘子；比如a=0时，那么就是纯粹的relu，我们记得纯粹的relu是需要乘以math.sqrt(2/n), 此时gain(0)=math.sqrt(2)，刚好也是契合的。

In [27]:
gain(1),gain(0),gain(0.01),gain(0.1),gain(math.sqrt(5.))

(1.0,
 1.4142135623730951,
 1.4141428569978354,
 1.4071950894605838,
 0.5773502691896257)

In [28]:
torch.zeros(10000).uniform_(-1,1).std()

tensor(0.5784)

uniform的variance是 (math.square(b-a))/12, 对于uniform (-1,1) 这里就是 1/3，那么std就是math.sqrt(1/3)

In [29]:
1/math.sqrt(3.)

0.5773502691896258

In [37]:
def kaiming2(x,a, use_fan_out=False):
    nf,ni,*_ = x.shape
    rec_fs = x[0,0].shape.numel()
    fan = nf*rec_fs if use_fan_out else ni*rec_fs
    std = gain(a) / math.sqrt(fan)
    bound = math.sqrt(3.) * std
    x.data.uniform_(-bound,bound) #这里让x也就是weight的standard deviation 等于 上面计算出来的std which is gain(a)/math.sqrt(fan)
    #之前讲过weight也是先初始化成vaiance=1的，然后乘以 gain()/math.sqrt(fan), 那么其standard deviation就是gain()/math.sqrt(fan)
    #####weight initlization的规则是 不管weight是什么分布，weight的standard deviation 需等于 gain(leakly_parameter)/math.sqrt(fan) ######
    ##我们看到，kaiming2确实做到了这个要求

In [32]:
kaiming2(l1.weight, a=0);
stats(f1(x))

(tensor(0.5123, grad_fn=<MeanBackward0>),
 tensor(0.9764, grad_fn=<StdBackward0>))

In [33]:
kaiming2(l1.weight, a=math.sqrt(5.))
stats(f1(x))

(tensor(0.2092, grad_fn=<MeanBackward0>),
 tensor(0.4375, grad_fn=<StdBackward0>))

上面看出，conv后面跟的relu的leaky程度跟weight initialization时乘以的那个gain值的leaky程度不一样，导致relu后方法剧烈变化了

In [34]:
kaiming2(l1.weight, a=math.sqrt(5.))
stats(f1(x,a=math.sqrt(5.)))

(tensor(-0.2607, grad_fn=<MeanBackward0>),
 tensor(1.0232, grad_fn=<StdBackward0>))

上面看出，conv后面跟的什么leaky程度的relu,那么得保证weight initialization时乘以的那个gain值的leaky程度一致，才能保证relu后的数据还是方差为1;或者说，方差能尽量保持住

In [38]:
class Flatten(nn.Module):
    def forward(self,x): return x.view(-1)

In [39]:
m = nn.Sequential(
    nn.Conv2d(1,8, 5,stride=2,padding=2), nn.ReLU(),
    nn.Conv2d(8,16,3,stride=2,padding=1), nn.ReLU(),
    nn.Conv2d(16,32,3,stride=2,padding=1), nn.ReLU(),
    nn.Conv2d(32,1,3,stride=2,padding=1),
    nn.AdaptiveAvgPool2d(1),
    Flatten(),
)

In [40]:
y = y_valid[:100].float()

In [41]:
t = m(x)
stats(t)

(tensor(-0.0095, grad_fn=<MeanBackward0>),
 tensor(0.0119, grad_fn=<StdBackward0>))

In [42]:
l = mse(t,y)
l.backward()

In [43]:
stats(m[0].weight.grad)

(tensor(-0.0426), tensor(0.0460))

In [44]:
init.kaiming_uniform_??

In [45]:
for l in m:
    if isinstance(l,nn.Conv2d):
        init.kaiming_uniform_(l.weight)
        l.bias.data.zero_()

In [46]:
t = m(x)
stats(t)

(tensor(-0.0264, grad_fn=<MeanBackward0>),
 tensor(0.3940, grad_fn=<StdBackward0>))

In [47]:
l = mse(t,y)
l.backward()
stats(m[0].weight.grad)

(tensor(0.0419), tensor(0.6745))

## Export

In [48]:
!./notebook2script.py 02a_why_sqrt5.ipynb

Converted 02a_why_sqrt5.ipynb to exp/nb_02a.py
